In [ ]:
!pip install 'descarteslabs[complete]'

In [ ]:
!python -V

In [ ]:
import os
import descarteslabs as dl 
from descarteslabs.client.services.tasks import AsyncTasks, as_completed
from descarteslabs.client.services.catalog import Catalog

In [ ]:
def rice_sar_stats(year, product_id, tile):

    import json
    import numpy as np
    import logging
    import time
    from datetime import datetime
    catalog = Catalog()

    def calculate_stats(arr):

        vx_max = arr.max(axis=0)
        vx_min = arr.min(axis=0)
        vx_mean = arr.mean(axis=0)
        vx_med = np.ma.median(arr, axis=0)
        vx_std = arr.std(axis=0)

        stats = [vx_max, vx_min, vx_mean, vx_med, vx_std]

        return stats
    
    def get_imagery(year, tile, s1_pass, band):
        
        start = str(year) + '-01-01'
        end = str(year) + '-12-31'
        
        dlkey = tile['properties']['key']
        
        scenes, geoctx = dl.scenes.search(
            tile['geometry'],
            products=["sentinel-1:GRD"],
            start_datetime=start,
            end_datetime=end,
            limit=1000
        )
        
        # scenes['features'] DNE...
#         if s1_pass in ['DESCENDING', 'ASCENDING']:
#             scenes['features'] = [
#                 ft for ft in scenes['features'] if ft['properties']['pass'] == s1_pass
#             ]

        geoctx = geoctx.assign(resolution=20)
        stack = scenes.stack(
            band,
            geoctx,
            mask_alpha=False
        )
        
        # https://docs.descarteslabs.com/descarteslabs/scenes/docs/scenecollection.html
        # returned array’s shape is (scene, band, y, x) if bands_axis is 1
        # only have 1 band, so take the first
        _stack = stack[:,0,:,:]
        return _stack, geoctx

    def catalog_upload(tile, stats_list, meta, year):
        # Upload the ndarray as a single scene in our new product
        t_props = tile['properties']
        
        catalog.upload_ndarray(
            stats_list,
            product_id=product_id,
            image_id=t_props['key'],
            proj4=t_props['proj4'],
            geotrans=t_props['geotrans']
        )
        return

    def run_analysis(year, tile):

        bands = [['vv'], ['vh']]
        passes = ['DESCENDING', 'ASCENDING', 'BOTH']
        all_stats = []

        for _ib, band in enumerate(bands):
            print(f"\tBAND {_ib + 1}/{len(bands)}: {band}")
            for _ip, s1_pass in enumerate(passes):
                print(f"\t\tPASS {_ip + 1}/{len(passes)}: {s1_pass}")
                
                try:
                    imagery, meta = get_imagery(year, tile, s1_pass, band)
                    arr = np.ma.masked_equal(imagery, 0)
                    stats = calculate_stats(arr)
                    all_stats.append(stats)
                except Exception as e:
                    print(e)
                    tile_size = tile['properties']['tilesize']
                    nodata = np.zeros((tile_size, tile_size), dtype=np.uint8)
                    all_stats.append([nodata, nodata, nodata, nodata, nodata])

#         # ---------------------
#         # TEST IT FASTER WITH ONE BAND
#         band = ['vv']
#         s1_pass = 'DESCENDING'
#         imagery, meta = get_imagery(year, tile, s1_pass, band)
#         print(meta)
#         arr = np.ma.masked_equal(imagery, 0)
#         stats = calculate_stats(arr)
#         all_stats.append(stats)
#         # ---------------------
         
        # flatten the list of lists (all_stats) into a single list
        stats_list = [item for sublist in all_stats for item in sublist]
        stats_ndarray = np.array(stats_list)
        
        # re-shape stats_ndarray as (x, y, band (?))
        stats_ndarray = np.transpose(stats_ndarray, (1,2,0))
        catalog_upload(tile, stats_ndarray, meta, year)
        return
    
    return run_analysis(year,tile)



In [ ]:
""" For testing
"""
year = '2017'
loc = 'singapore'
    
res = 20.0
pad = 0
pixels = 1024

aoi = dl.places.shape(loc, geom = 'low')
tiles = dl.raster.dltiles_from_shape(res, pixels, pad, aoi['geometry'])

# get username to generate product_id
user = os.environ.get('USER')

product_id = '{}:{}:test'.format(
    dl.Auth().namespace,
    ''.join(user.lower().split(' '))
)

for _it, tile_ft in enumerate(tiles['features']):

    print(f"\nTILE {_it + 1}/{len(tiles['features'])}\n")
    
    rice_sar_stats(
        year,
        product_id,
        tile_ft
    )

In [ ]:
year = '2017'
loc = 'singapore'
    
res = 20.0
pad = 0
pixels = 1024 

aoi = dl.places.shape(loc, geom = 'low')
tiles = dl.raster.dltiles_from_shape(res, pixels, pad, aoi['geometry'])

# get username to generate product_id
user = os.environ.get('USER')
    
product_id = '{}:{}:test'.format(
    dl.Auth().namespace,
    ''.join(user.lower().split(' '))
)


try:
    at = AsyncTasks()
    async_function = at.create_function(
            rice_sar_stats, cpus=1, memory="6Gi", name="sar_img_stats",
            maximum_concurrency = 14,
            image='us.gcr.io/dl-ci-cd/images/tasks/public/py3.7/default:v2018.11.27')
    # Iterate over tiles, submit tasks
#     tasks = []
#     for tile in tiles['features']:

#         print("TILE ID: %s" % tile['properties']['key'])

#         t = async_function( 
#             year,
#             product_id,
#             tile,
#         )
#         tasks.append(t)

    # make it quick => only one tile
    tasks = [async_function(
        year,
        product_id,
        tiles['features'][0]
    )]

    print('Done submitting %i tasks. Starting to collect' % len(tasks))

    for task in as_completed(tasks):

        if task.exception:
            print("Failed with exception: ", task.log)
        else:
            task.result
except:
    print ("tasks not submitted!")
    